In [ ]:
!pip install torchxrayvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 52.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torchxrayvision as xrv
import skimage, torch, torchvision


# Prepare the image:
def create_findings(image_path):
  img = skimage.io.imread(image_path)
  # img = skimage.io.imread("image.png")
  # img = skimage.io.imread("CXR1_1_IM-0001-3001.png")
  img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
  img = img.mean(2)[None, ...] # Make single color channel

  transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

  img = transform(img)
  img = torch.from_numpy(img)

  # Load model and process image
  model = xrv.models.DenseNet(weights="densenet121-res224-all")
  outputs = model(img[None,...]) # or model.features(img[None,...])

  # Print results
  print(dict(zip(model.pathologies,outputs[0].detach().numpy())))
  data = dict(zip(model.pathologies,outputs[0].detach().numpy()))

  high_chance_conditions = [condition for condition, probability in data.items() if probability >= 0.75]
  moderate_chance_conditions = [condition for condition, probability in data.items() if 0.6 <= probability < 0.75]


  # Generate the summary
  summary_text = "The imaging analysis reveals "
  if high_chance_conditions or moderate_chance_conditions:
      if high_chance_conditions:
          summary_text += f"a high likelihood for the following conditions: {', '.join(high_chance_conditions)},\n"
      if moderate_chance_conditions:
          summary_text += f"and a moderate likelihood for the following conditions: {', '.join(moderate_chance_conditions)}.\n"
  else:
      summary_text = "No significant abnormalities detected from the imaging analysis. Findings appear within normal limits."

  # Output the summary
  # print("Generated Summary:")
  # print(summary_text)
  return summary_text

# {'Atelectasis': 0.32797316,
#  'Consolidation': 0.42933336,
#  'Infiltration': 0.5316924,
#  'Pneumothorax': 0.28849724,
#  'Edema': 0.024142697,
#  'Emphysema': 0.5011832,
#  'Fibrosis': 0.51887786,
#  'Effusion': 0.27805611,
#  'Pneumonia': 0.18569896,
#  'Pleural_Thickening': 0.24489835,
#  'Cardiomegaly': 0.3645515,
#  'Nodule': 0.68982,
#  'Mass': 0.6392845,
#  'Hernia': 0.00993878,
#  'Lung Lesion': 0.011150705,
#  'Fracture': 0.51916164,
#  'Lung Opacity': 0.59073937,
#  'Enlarged Cardiomediastinum': 0.27218717}


/usr/local/lib/python3.10/dist-packages/torchxrayvision/utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
!pip install transformers

In [ ]:

from transformers import pipeline

classifier = pipeline(model="lxyuan/vit-xray-pneumonia-classification")

# image taken from https://www.news-medical.net/health/What-is-Viral-Pneumonia.aspx
# classifier("/content/gen/pneumonia_0.png")


Device set to use cpu


[{'label': 'PNEUMONIA', 'score': 0.9050011038780212}, {'label': 'NORMAL', 'score': 0.10924490541219711}]


In [ ]:

count=0
true=0
folder = "/content/drive/MyDrive/NLMCXR/DCGAN_images/DCGAN/"

# img_path="/content/drive/MyDrive/NLMCXR/ACGAN/pneumonia_0.png"
# output=classifier(img_path)
# print(output[0]['score'])

#for all files in the folder
for filename in os.listdir(folder):
  if filename.endswith(".png"):
    count+=1
    img_path = os.path.join(folder, filename)
    print(count)
    output=classifier(img_path)
    if output[0]['score']>=0.8:
      true+=1

print(true/count)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
#for each image in gen folder, use classifier
folder = "/content/gen/"

# Loop through all files in the folder
for filename in os.listdir(folder):
  if filename.endswith(".png"):
    img_path = os.path.join(folder, filename)
    print(img_path)
    classifier(img_path)

/content/gen/pneumonia_3.png
/content/gen/pneumonia_9.png
/content/gen/pneumonia_8.png
/content/gen/pneumonia_0.png
/content/gen/pneumonia_7.png
/content/gen/pneumonia_2.png
/content/gen/pneumonia_1.png


**FID evaluation**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch-fid

In [ ]:
#make images in real folder 112x112
from PIL import Image
import os

# Set the path to your "real" folder
real_folder = "/content/real"

# Loop through all files in the folder
for filename in os.listdir(real_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):  # Check if it's an image file
        # Open the image using Pillow
        img_path = os.path.join(real_folder, filename)
        img = Image.open(img_path)

        # Resize the image to 112x112
        resized_img = img.resize((112, 112))

        # Save the resized image back to the "real" folder, overwriting the original
        resized_img.save(img_path)

print("Resizing complete.")




Resizing complete.


In [ ]:
from PIL import Image
import os

# Define the folder path
folder_path = "/content/real1"
output_size = (64,64)

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Check if the file is an image
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
        try:
            # Open the image
            with Image.open(file_path) as img:
                # Resize the image
                img_resized = img.resize(output_size)

                # Save the resized image (overwrite the original file)
                img_resized.save(file_path)
                print(f"Resized: {filename}")
        except Exception as e:
            print(f"Error resizing {filename}: {e}")


Resized: person1001_bacteria_2932.jpeg
Resized: person1004_bacteria_2935.jpeg
Resized: person1003_bacteria_2934.jpeg
Resized: person1000_bacteria_2931.jpeg
Resized: person1002_bacteria_2933.jpeg


In [ ]:
!python -m pytorch_fid "/content/real1" "/content/gen"

100% 1/1 [00:02<00:00,  2.53s/it]
100% 1/1 [00:02<00:00,  2.30s/it]
FID:  384.87873746828126
